In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Input, Flatten, Dot
from tensorflow.keras.models import Model

In [7]:
# Open the file in binary mode, remove null bytes, and then decode
with open('G:\\Mi unidad\\SOCOFAR\\0.-Recomendadores-Oficial\\Archivos\\SR_BASE_FINAL_PROD_FEB24.csv', 'rb') as file:
    content = file.read().replace(b'\x00', b'')  # Replace null bytes with nothing

# Convert the binary content to a string and use StringIO to mimic a file object for pandas
from io import StringIO
content_str = StringIO(content.decode('latin'))

# Now use pd.read_csv on the cleaned string data
df = pd.read_csv(content_str, sep=';', encoding='latin')
df['RUT_CLIENTE']=((df['SK']+1599)/2).astype(int)

In [10]:
# Cargar tu conjunto de datos desde un archivo CSV o una fuente de datos
# Ejemplo de formato de datos en un DataFrame:
# df = pd.read_csv("tu_archivo.csv")
# Debe contener columnas "rut", "producto" y "cantidad"

df['SUM_NRO_PROD'] = df['SUM_NRO_PROD'].fillna(1.0)
df=df[['RUT_CLIENTE','CODIGO_PRODUCTO','SUM_NRO_PROD']].head(1000000)

# Crear un mapeo de IDs para usuarios y productos
user_ids = df["RUT_CLIENTE"].unique()
product_ids = df["CODIGO_PRODUCTO"].unique()

user_id_map = {id: i for i, id in enumerate(user_ids)}
product_id_map = {id: i for i, id in enumerate(product_ids)}

# Agregar columnas de IDs de usuario y producto al DataFrame
df["user_id"] = df["RUT_CLIENTE"].map(user_id_map)
df["product_id"] = df["CODIGO_PRODUCTO"].map(product_id_map)

In [ ]:
# Dividir el conjunto de datos en entrenamiento y prueba
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Definir las capas de entrada
user_input = Input(shape=(1,))
product_input = Input(shape=(1,))

# Definir las capas de embedding para usuarios y productos
user_embedding = Embedding(len(user_ids), 50)(user_input)
product_embedding = Embedding(len(product_ids), 50)(product_input)

# Calcular la similitud entre usuarios y productos utilizando el producto punto
dot_product = Dot(axes=2)([user_embedding, product_embedding])

In [ ]:
# Crear el modelo recomendador
model = Model(inputs=[user_input, product_input], outputs=dot_product)

# Compilar el modelo
model.compile(loss='mean_squared_error', optimizer='adam')

# Entrenar el modelo
model.fit([train_df["user_id"], train_df["product_id"]], train_df["SUM_NRO_PROD"], epochs=10, batch_size=64)

In [ ]:
# Evaluar el modelo en el conjunto de prueba
loss = model.evaluate([test_df["user_id"], test_df["product_id"]], test_df["SUM_NRO_PROD"])
print(f"Pérdida en el conjunto de prueba: {loss}")

# Realizar predicciones para un usuario dado
user_id_to_predict = 0  # Cambia esto al ID del usuario que deseas recomendar
product_ids_to_predict = list(product_id_map.values())
predictions = model.predict([np.array([user_id_to_predict] * len(product_ids_to_predict)), np.array(product_ids_to_predict)])
predicted_ratings = predictions.flatten()

# Obtener los productos recomendados ordenados por puntaje
recommendations = [(product_ids[i], predicted_ratings[i]) for i in np.argsort(predicted_ratings)[::-1]]
print("Productos recomendados:")
for product_id, score in recommendations:
    print(f"Producto: {product_id}, Puntuación: {score}")

In [1]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple/
     ------------------------------------- 300.9/300.9 MB 11.3 MB/s eta 0:00:00
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ---------------------------------------- 2.7/2.7 MB 85.9 MB/s eta 0:00:00
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl (24.4 MB)
     ------------------------------------- 938.7/938.7 kB 61.9 MB/s eta 0:00:00
     --------------------------------------- 15.8/15.8 MB 43.7 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ------------------------------------- 413.4/413.4 kB 25.2 MB/s eta 0:00:00
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
     ---------------------------------------- 1.5/1.5 MB 47.6 MB/s eta 0:00:00
     

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.9.3 requires numpy<1.26.0,>=1.18.5, but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
python.exe -m pip install --upgrade pip